In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [3]:
import chromadb

In [4]:
model_local = ChatOllama(model="mistral")

docs_paths = ["./National_Efficient_Price_Determination_2022_23.pdf", 
              "./National_Efficient_Price_Determination_2023_24.pdf"]

all_doc_splits = []

for path in docs_paths:
    loader = PyPDFLoader(path)
    doc_splits = loader.load_and_split()
    all_doc_splits.extend(doc_splits)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=100) 
chunk_doc_splits = text_splitter.split_documents(all_doc_splits)

In [ ]:
embeddings = OllamaEmbeddings(model='nomic')
db = FAISS.from_documents(chunk_doc_splits, embeddings)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [5]:
vectorstore = Chroma.from_documents(
    documents=chunk_doc_splits,
    collection_name="rag-chroma",
    embedding=OllamaEmbeddings(model='nomic'),
)
retriever = vectorstore.as_retriever()

In [ ]:
before_rag_template = "What is {topic}"
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic": "Nep"}))

In [ ]:
after_rag_template = """Answer the question based only on the following context:
{context}. If it's out of your ability to answer the question, don't answer with falsification of facts and apologize for the inconvenience.
Provide a refernece of in which file which page you found the answer at the end.
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("Does Lens Interventions have a same day payment list in 2023-24?"))

In [ ]:
conversation_history = []

after_rag_template = """Answer the question based on the following context: {context},
                        and previous history: {history}.
                        If it's out of your ability to answer the question, don't answer with falsification of facts and apologize for the inconvenience.
                        Provide a reference of in which file and which page you found the answer at the end.
                        Question: {question}
                        """

In [ ]:
def format_history_for_inclusion(history):     
    formatted_history = ""
    for turn in history:         
        formatted_history += f"{turn['role']}: {turn['content']}\n"
    return formatted_history

In [ ]:
def invoke_with_history(question):
    global conversation_history
    formatted_history = format_history(conversation_history)

    after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
    after_rag_chain = (
        {"context":retriever, "question":RunnablePassthrough(), "history": formatted_history}
        | after_rag_prompt
        | model_local
        | StrOutputParser()
    )

    response = after_rag_chain.invoke(question)

    conversation_history.append({"role": "user", "content": question})
    conversation_history.append({"role": "assistant", "content": response})

    return response

In [ ]:
print(invoke_with_history("Hello, my name is Olaf"))